In [19]:
import requests
import json
import sys

sys.path.append('..')

# Data Manipulation
import pandas as pd

from utils import *

# Agregados

In [20]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados'
df_aggregated = make_df(get_request_json(url))
df_aggregated

,id,nome,agregados
0,D5,Áreas Urbanizadas,"[{'id': '8418', 'nome': 'Áreas urbanizadas, Lo..."
1,CL,Cadastro Central de Empresas,"[{'id': '1732', 'nome': 'Dados gerais das empr..."
2,CA,Censo Agropecuário,"[{'id': '1278', 'nome': 'Agroindústria rural n..."
3,ME,"Censo Comum do Mercosul, Bolívia e Chile","[{'id': '2059', 'nome': 'Domicílios e Populaçã..."
4,CD,Censo Demográfico,"[{'id': '1301', 'nome': 'Área e Densidade demo..."
...,...,...,...
63,VS,Produção da Extração Vegetal e da Silvicultura,"[{'id': '5930', 'nome': 'Área total existente ..."
64,PO,Produção de Ovos de Galinha,"[{'id': '915', 'nome': 'Número de informantes,..."
65,IO,Produto Interno Bruto dos Municípios,"[{'id': '599', 'nome': 'Índice de Gini do prod..."
66,XE,Projeção da População,"[{'id': '7362', 'nome': 'Esperança de vida ao ..."


In [21]:
research_ids = df_aggregated['id'].unique()
print(list(research_ids))

['D5', 'CL', 'CA', 'ME', 'CD', 'CM', 'DU', 'DT', 'C4', 'DR', 'DZ', 'SU', 'ST', 'DE', 'D9', 'XF', 'FP', 'IU', 'IL', 'IR', 'IJ', 'PC', 'IA', 'IQ', 'IG', 'PG', 'LA', 'C2', 'AC', 'PB', 'PY', 'PP', 'AM', 'ES', 'PM', 'IT', 'OF', 'SH', 'SE', 'PT', 'RC', 'PI', 'PK', 'PJ', 'DG', 'PZ', 'XY', 'AB', 'MC', 'PE', 'SC', 'PL', 'SB', 'XN', 'AA', 'PD', 'B5', 'BB', 'DD', 'AX', 'CQ', 'LT', 'PA', 'VS', 'PO', 'IO', 'XE', 'SI']


In [22]:
%%time
df_researches = pd.DataFrame()

# select only the census data
df_aggregated = df_aggregated[df_aggregated['id'] == 'CD']

# select the unique research ids
research_ids = df_aggregated['id'].unique()

for research_id in research_ids:
    # get just the row for the given research id
    df_research = df_aggregated[df_aggregated['id'] == research_id]
    # explode the row to get all the agregados 
    df_research = df_research.explode('agregados')
    # reset index
    df_research = df_research.reset_index()
    # normalize the agregados column and rename the columns
    df_normalize = pd.json_normalize(df_research['agregados'])
    df_normalize = df_normalize.rename(columns = {'id': 'id-agregado', 'nome': 'nome-agregado'})
    # concat the agregado data to df_research
    df_research = pd.concat([df_research, df_normalize], axis = 1)
    # delete the dict column
    df_research = df_research.drop('agregados', axis = 1)
    # rename the columns
    df_research = df_research.rename(columns = {'id': 'id-pesquisa', 'nome': 'nome-pesquisa'})
    # concat the research data to df_researches
    df_researches = pd.concat([df_researches, df_research], axis = 0)
# reset index
df_researches = df_researches.reset_index(drop = True)

CPU times: total: 15.6 ms
Wall time: 9 ms


In [23]:
df_researches

,index,id-pesquisa,nome-pesquisa,id-agregado,nome-agregado
0,4,CD,Censo Demográfico,1301,Área e Densidade demográfica da unidade territ...
1,4,CD,Censo Demográfico,617,Brasileiros natos por unidade da federação de ...
2,4,CD,Censo Demográfico,2149,Brasileiros natos por unidade da federação de ...
3,4,CD,Censo Demográfico,164,Chefes de domicílios particulares permanentes ...
4,4,CD,Censo Demográfico,171,Chefes de domicílios particulares permanentes ...
...,...,...,...,...,...
933,4,CD,Censo Demográfico,241,Valor do rendimento nominal médio mensal dos d...
934,4,CD,Censo Demográfico,3283,Valor do rendimento nominal médio mensal e Val...
935,4,CD,Censo Demográfico,3282,Valor do rendimento nominal médio mensal e Val...
936,4,CD,Censo Demográfico,3527,Valor do rendimento nominal médio mensal e Val...


# Metadados das pesquisas

In [24]:
%%time
df_metadata = pd.DataFrame()

aggregate_ids = df_researches['id-agregado'].unique()

for aggregate_id in aggregate_ids:
    # create the url to request the metadata
    url_metadata = f"https://servicodados.ibge.gov.br/api/v3/agregados/{aggregate_id}/metadados"
    # make the request and get the metadata json
    df_aux = make_df([get_request_json(url_metadata)])
    # explode the row to get all the metadata
    # df_aux = df_aux.explode(['metadados'])
    # concat the metadata to df_metadata
    df_metadata = pd.concat([df_metadata, df_aux], axis = 0)
    # convert the floats to int
# reset index
df_metadata = df_metadata.reset_index(drop = True)

In [ ]:
df_metadata.info()
df_metadata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


""


In [8]:
# checar mais tarde
df_metadata['classificacoes'][2]

KeyError: 'classificacoes'

In [ ]:
df_metadata['variaveis'][2]

[{'id': 706,
  'nome': 'Número de unidades locais',
  'unidade': 'Unidades',
  'sumarizacao': []},
 {'id': 1000706,
  'nome': 'Número de unidades locais - percentual do total geral',
  'unidade': '%',
  'sumarizacao': []},
 {'id': 707,
  'nome': 'Pessoal ocupado total',
  'unidade': 'Pessoas',
  'sumarizacao': []},
 {'id': 1000707,
  'nome': 'Pessoal ocupado total - percentual do total geral',
  'unidade': '%',
  'sumarizacao': []},
 {'id': 708,
  'nome': 'Pessoal ocupado assalariado',
  'unidade': 'Pessoas',
  'sumarizacao': []},
 {'id': 1000708,
  'nome': 'Pessoal ocupado assalariado - percentual do total geral',
  'unidade': '%',
  'sumarizacao': []},
 {'id': 709, 'nome': 'Salários', 'unidade': 'Mil Reais', 'sumarizacao': []},
 {'id': 1000709,
  'nome': 'Salários - percentual do total geral',
  'unidade': '%',
  'sumarizacao': []}]

## Testes

+ para cada agregado
+ para cada nível
+ explodir os resultados
+ para cada variável
+ fazer um df
+ transformar a coluna de série em um dataframe

In [ ]:
agregado = 1734
nivel = 'N2' #[N2, N6, N3]	[N23, N132]
url = f'https://servicodados.ibge.gov.br/api/v3/agregados/{agregado}/variaveis?localidades={nivel}[all]' 

In [ ]:
make_df(get_request_json(url))

,id,variavel,unidade,resultados
0,706,Número de unidades locais,Unidades,"[{'classificacoes': [{'id': '319', 'nome': 'Fa..."
1,707,Pessoal ocupado total,Pessoas,"[{'classificacoes': [{'id': '319', 'nome': 'Fa..."
2,708,Pessoal ocupado assalariado,Pessoas,"[{'classificacoes': [{'id': '319', 'nome': 'Fa..."
3,709,Salários,Mil Reais,"[{'classificacoes': [{'id': '319', 'nome': 'Fa..."


In [ ]:
make_df(get_request_json(url))['resultados'][3]

[{'classificacoes': [{'id': '319',
    'nome': 'Faixas de pessoal ocupado',
    'categoria': {'0': 'Total'}},
   {'id': '11939',
    'nome': 'Classificação Nacional de Atividades Econômicas (CNAE)',
    'categoria': {'0': 'Total'}}],
  'series': [{'localidade': {'id': '1',
     'nivel': {'id': 'N2', 'nome': 'Grande Região'},
     'nome': 'Norte'},
    'serie': {'2001': '..',
     '2002': '..',
     '2003': '..',
     '2004': '..',
     '2005': '..',
     '2006': '..'}},
   {'localidade': {'id': '2',
     'nivel': {'id': 'N2', 'nome': 'Grande Região'},
     'nome': 'Nordeste'},
    'serie': {'2001': '..',
     '2002': '..',
     '2003': '..',
     '2004': '..',
     '2005': '..',
     '2006': '..'}},
   {'localidade': {'id': '3',
     'nivel': {'id': 'N2', 'nome': 'Grande Região'},
     'nome': 'Sudeste'},
    'serie': {'2001': '..',
     '2002': '..',
     '2003': '..',
     '2004': '..',
     '2005': '..',
     '2006': '..'}},
   {'localidade': {'id': '4',
     'nivel': {'id': 'N2', '

In [ ]:
make_df(get_request_json(url)).explode('resultados').explode('resultados')

,id,variavel,unidade,resultados
0,706,Número de unidades locais,Unidades,classificacoes
0,706,Número de unidades locais,Unidades,series
1,707,Pessoal ocupado total,Pessoas,classificacoes
1,707,Pessoal ocupado total,Pessoas,series
2,708,Pessoal ocupado assalariado,Pessoas,classificacoes
2,708,Pessoal ocupado assalariado,Pessoas,series
3,709,Salários,Mil Reais,classificacoes
3,709,Salários,Mil Reais,series


In [ ]:
make_df(make_df(get_request_json(url)).explode('resultados')['resultados'][0]['series'])

,localidade_id,nivel_id,nivel_nome,localidade_nome,serie_2001,serie_2002,serie_2003,serie_2004,serie_2005,serie_2006
0,1,N2,Grande Região,Norte,165881,179327,191288,199119,214831,216147
1,2,N2,Grande Região,Nordeste,764344,817604,855239,888667,952708,959192
2,3,N2,Grande Região,Sudeste,2570261,2719949,2830450,2915695,3057349,3101342
3,4,N2,Grande Região,Sul,1187737,1246216,1302837,1351186,1416751,1433972
4,5,N2,Grande Região,Centro-Oeste,353992,376543,392249,411260,435301,433847


In [ ]:
make_df(make_df(get_request_json(url)).explode('resultados')['resultados'][0]['classificacoes'])

,id,nome,categoria_0
0,319,Faixas de pessoal ocupado,Total
1,11939,Classificação Nacional de Atividades Econômica...,Total
